In [2]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
import pickle

# Category

In [147]:
df= pd.read_csv('Data.csv')
df = df.sort_values('Location')

In [4]:
df.head()

,Location,Social Category,PrimaryGirls,PrimaryBoys,PrimaryOverall,UpperGirls,UpperBoys,UpperOverall,SecondGirls,SecondBoys,SecondOverall
0,AHMEDABAD,General,0.00,0.00,0.00,0.00,0.00,0.00,8.02,12.34,10.54
633,AHMEDABAD,OBC,18.12,18.26,18.19,26.75,19.81,22.98,35.95,39.83,38.35
632,AHMEDABAD,General,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.50,0.42
764,AHMEDABAD,General,9.23,12.54,11.02,12.00,12.73,12.40,12.77,21.93,18.05
765,AHMEDABAD,OBC,0.00,0.00,0.00,0.43,0.00,0.00,16.72,24.65,21.60


In [5]:
def find(overall,girl,boy):
    x = df[[overall]]
    y = df[[girl,boy]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=12,test_size=0.25)
    lr = LinearRegression()
    lr.fit(x_train,y_train)
    prediction = lr.predict(x_test)
    lr.score(x_test,y_test)
    pickle.dump(lr, open('model.pkl', 'wb'))
    pickled_model = pickle.load(open('model.pkl', 'rb'))
    pickled_model.predict(x_test)
    
    return [metrics.r2_score(y_test,prediction),metrics.mean_absolute_error(y_test,prediction) ]

In [6]:
primary = find('PrimaryOverall','PrimaryGirls', 'PrimaryBoys')

In [7]:
upper = find('UpperOverall','UpperGirls', 'UpperBoys')

In [8]:
second = find('SecondOverall','SecondGirls', 'SecondBoys')

# Cast Wise Prediction 

In [11]:
cast = df.sort_values('Social Category')

In [12]:
tempcast = cast['PrimaryBoys'] + cast['PrimaryGirls'] + cast['UpperBoys'] + cast['UpperGirls'] + cast['SecondBoys'] + cast['SecondGirls']
tempcastoverall = cast['PrimaryOverall'] + cast['UpperOverall'] + cast['SecondOverall']
Location = df['Location']
tempcat = df['Social Category']

In [13]:
cast = pd.DataFrame(zip(Location,tempcat,tempcast,tempcastoverall) , columns=['Loc' ,'Social Category','All Boys and Girls' , 'Overall'])

In [14]:
tempgeneral = cast[cast['Social Category']== 'General' ]['All Boys and Girls'].tolist()
tempgeneraloverall = cast[cast['Social Category'] == 'General' ]['Overall'].tolist()

tempobc = cast[cast['Social Category']== 'OBC' ]['All Boys and Girls'].tolist()
tempobcoverall = cast[cast['Social Category'] == 'OBC' ]['Overall'].tolist()

tempsc = cast[cast['Social Category']== 'SC' ]['All Boys and Girls'].tolist()
tempscoverall = cast[cast['Social Category'] == 'SC' ]['Overall'].tolist()

tempst = cast[cast['Social Category']== 'ST' ]['All Boys and Girls'].tolist()
tempstoverall = cast[cast['Social Category'] == 'ST' ]['Overall'].tolist()

In [119]:
cast.head(2)

,general,generaloverall,obc,obcoverall,sc,scoverall,st,stoverall
0,20.36,10.54,33.51,17.11,36.80,14.96,45.30,22.61
1,41.10,20.74,33.06,16.89,29.19,14.26,27.79,13.63


In [156]:
def Cast_acc(data,castoverall):
    y = cast[[data]]
    x = cast[[castoverall]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1, test_size=0.78)
    lr = LinearRegression()
    lr.fit(x_train,y_train)
    x1 = lr.predict(x_test)
    return [metrics.r2_score(x1,y_test),metrics.mean_absolute_error(x1,y_test)]

In [157]:
general = Cast_acc('general' , 'generaloverall')
obc = Cast_acc('obc' ,'obcoverall')
sc = Cast_acc("sc", 'scoverall')
st = Cast_acc('st','stoverall')

In [160]:
df2.to_csv('Linermodelaccuracy.csv')

# Gender Wise Data anlysis

In [153]:
# Temporary dataframe created using Original Data
temp_gender = df[[ 'Location' ,'PrimaryBoys' , 'PrimaryGirls' , 'UpperBoys', 'UpperGirls' ,'SecondBoys' ,'SecondGirls' ,'PrimaryOverall' , 'UpperOverall','SecondOverall']]  

# tempboy and tempfirl creat the all data
tempboy = temp_gender['PrimaryBoys'] + temp_gender['UpperBoys'] + temp_gender['SecondBoys']
tempgirl = temp_gender['PrimaryGirls'] + temp_gender['UpperGirls'] + temp_gender['SecondGirls']
tempoverall = temp_gender['PrimaryOverall'] + temp_gender['UpperOverall'] + temp_gender['SecondOverall']

# final Data Frame
final_gender = pd.DataFrame(zip(tempboy,tempgirl,tempoverall) , columns=['boys' ,'girls' , 'overall'])

In [155]:
final_gender.head(3)

,boys,girls,overall
0,12.34,8.02,10.54
1,77.90,80.82,79.52
2,1.50,0.00,0.42


In [166]:
def Gender():
    y = final_gender[['boys' , 'girls']]
    x = final_gender[['overall']]
    x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=12, test_size=0.25)
    
    lr = LinearRegression()
    lr.fit(x_train,y_train)
    pred = lr.predict(x_test)
    return [metrics.r2_score(pred,y_test),metrics.mean_absolute_error(pred,y_test)]

In [168]:
gender = Gender()

In [207]:
df2 = pd.DataFrame(zip(general,obc,sc,st,primary,upper,second,gender),columns=['general','obc','sc','st','primart','secondary','highersecondary','male/female']).T
df2.rename(columns={0 : 'R2_score' , 1: 'MAE'} , inplace = True)

In [208]:
df2.to_csv('Linermodelaccuracy.csv')